Download links:
- Attribution : https://zenodo.org/record/8202241 (please use version 1.2 with the Black Beetle & Wind merge)
- Year : https://zenodo.org/record/7080016#.Y7QtTS8w30o 

I recommand to check both maps and their CRS with QGIS before running this script. 

Note : time for each cell are indicated for a MacBook Pro with intel i5 processor. 

# Method :

1. Both rasters are loaded and transformed to GeoDataFrames. The polygon per raw created for both GeoDataFrames are not the same. We cannot join on INDEX. 
2. The two GeoDataFrames are spatially joined. This will ensure a correct spatial correspondance between date and cause from the two maps. 
3. Polygons are cleaned. First polygons are grouped per class and cause. Then, polygons are normalized and united together. Finally cluster of polygons with same date and cause are exploded to create a new polygon per row. 

In [1]:
#Loading both maps (~1min)
import rasterio
from rasterio.crs import CRS

import geopandas as gpd
import numpy as np 

directory_cause = '../data/SenfSeidl_maps/fire_wind_barkbeetle_france.tif'
#sometimes CRS is missing... Senf & Seidl use EPSG:3035 for the attribution. 
with rasterio.open(directory_cause, 'r+') as rds:
    rds.crs = CRS.from_epsg(3035)

src_cause = rasterio.open(directory_cause)
data_cause = src_cause.read(1)
print('attribution :')
print('-crs :', src_cause.crs)
print('-dtype :', src_cause.dtypes)
print('-nodata:', src_cause.nodata)
print('-values:', np.unique(data_cause))

directory_year = '../data/SenfSeidl_maps/france/disturbance_year_1986-2020_france.tif'
src_year = rasterio.open(directory_year)
data_year = src_year.read(1)
print('year :')
print('-crs :', src_year.crs)
print('-dtype :', src_year.dtypes)
print('-nodata:', src_year.nodata)
print('-values:', np.unique(data_year))

attribution :
-crs : EPSG:3035
-dtype : ('float32',)
-nodata: -3.3999999521443642e+38
-values: [-3.4e+38  1.0e+00  2.0e+00  3.0e+00]
year :
-crs : PROJCS["unnamed",GEOGCS["GRS 1980(IUGG, 1980)",DATUM["unknown",SPHEROID["GRS80",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
-dtype : ('uint16',)
-nodata: 65535.0
-values: [ 1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997
  1998  1999  2000  2001  2002  2003  2004  2005  2006  2007  2008  2009
  2010  2011  2012  2013  2014  2015  2016  2017  2018  2019  2020 65535]


In [2]:
#Converting to GeoDataFrame (~8min)

from rasterio.features import shapes
from shapely.geometry import shape

#masking nodata
import numpy.ma as ma 

mask = (data_year == 65535) 
data_year_ma = ma.masked_array(data_year, mask=mask)

mask = (data_cause < 0)
data_cause_ma = ma.masked_array(data_cause, mask=mask)

#map with year of disturbance + conversion to CRS EPSG:3035
shape_gen = ((shape(s), int(v)) for s,v in  shapes(data_year_ma, transform=src_year.transform))
gdf1 = gpd.GeoDataFrame(dict(zip(["geometry", "year"], zip(*shape_gen))), crs=src_year.crs).to_crs(src_cause.crs)

#map with cause of disturbance 
shape_gen = ((shape(s), int(v)) for s,v in  shapes(data_cause_ma, transform=src_cause.transform))
gdf2 = gpd.GeoDataFrame(dict(zip(["geometry", "year"], zip(*shape_gen))), crs=src_cause.crs)

In [7]:
#Joining both GeoDataFrames (~2min)

import dask_geopandas as dgpd 

dgdf1 = dgpd.from_geopandas(gdf1, npartitions=10)
dgdf2 = dgpd.from_geopandas(gdf2, npartitions=10)

dgdf = dgpd.sjoin(dgdf1, dgdf2, how='inner', op='intersects').compute()
dgdf.rename(columns={'year_left':'year', 'year_right':'cause'}, inplace=True)
dgdf.reset_index(inplace=True)
dgdf.head()


/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1127/3739704228.py:8: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  dgdf = dgpd.sjoin(dgdf1, dgdf2, how='inner', op='intersects').compute()


,index,geometry,year,index_right,cause
0,0,"POLYGON ((3798746.638 3134740.558, 3798746.638...",2018,0,3
1,1,"POLYGON ((3798836.638 3134740.558, 3798836.638...",2018,1,3
2,2,"POLYGON ((3799226.638 3134680.558, 3799226.638...",2004,2,3
3,3,"POLYGON ((3798776.638 3134620.558, 3798776.638...",2004,3,1
4,4,"POLYGON ((3798866.638 3134590.558, 3798896.638...",2004,3,1


In [27]:
gdf = dgdf.compute()
gdf.to_parquet('../data/SenfSeidl_maps/dgdf.parquet')

In [8]:
#Cleaning the Joined GeoDataFrame (~130min)

#Defining a variable for grouping rows with the same cause and year. Because we want to group polygons nearby with the same cause and year.
def get_similar(row):
    cells = [str(row.cause), str(row.year)]
    return '-'.join(cells)

dgdf['similar'] = dgdf.apply(get_similar, axis=1)

# Grouping by similar 
dgdf = dgpd.from_geopandas(dgdf, npartitions=10) #not sure if we need to use Dask here
gdf_grouped = dgdf.dissolve(by='similar', aggfunc='mean').compute()


In [12]:
#normalize the polygon -> This method orders the coordinates, rings of a polygon and parts of multi geometries consistently. (critical for next step)
gdf_grouped.geometry = gdf_grouped.geometry.normalize()

In [13]:
#union the polygons -> Transform list of polygon into a MultiPolygons 
#~ 70min
from shapely.ops import unary_union
gdf_grouped.geometry = gdf_grouped.geometry.apply(unary_union)

/Users/arthurcalvi/Venv/DiAtDaJo/lib/python3.9/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [14]:
gdf_exploded = gdf_grouped.explode()
print('After explode, shape : ', gdf_exploded.shape)
print(gdf_exploded.head())

/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1127/1857334894.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_exploded = gdf_grouped.explode()


After explode, shape :  (5525515, 5)
                  index    year    index_right  cause  \
similar                                                 
3-2018  0  1.913211e+06  2018.0  860217.130041    3.0   
        1  1.913211e+06  2018.0  860217.130041    3.0   
        2  1.913211e+06  2018.0  860217.130041    3.0   
        3  1.913211e+06  2018.0  860217.130041    3.0   
        4  1.913211e+06  2018.0  860217.130041    3.0   

                                                    geometry  
similar                                                       
3-2018  0  POLYGON ((3327146.638 2782150.558, 3327146.638...  
        1  POLYGON ((3327116.638 2782210.558, 3327146.638...  
        2  POLYGON ((3326576.638 2782600.558, 3326666.638...  
        3  POLYGON ((3345626.638 2807350.558, 3345626.638...  
        4  POLYGON ((3345596.638 2807410.558, 3345626.638...  


In [ ]:
#Cleaning the Joined GeoDataFrame (~130min)
def get_similar(row):
    cells = [str(row.cause), str(row.year)]
    return '-'.join(cells)
    
dgdf['similar'] = dgdf.apply(get_similar, axis=1)

#group by year and cause. 
# ~ 130min
gdf_grouped = dgdf.dissolve(by='similar', aggfunc='mean')
print('After regrouping by year and chause, shape : ', gdf_grouped.shape)
print(gdf_grouped.head())

#normalize the polygon -> This method orders the coordinates, rings of a polygon and parts of multi geometries consistently. (critical for next step)
# ~ 1min
gdf_grouped.geometry = gdf_grouped.geometry.normalize()

#union the polygons -> Transform list of polygon into a MultiPolygons 
# ~ 1 hours 
from shapely.ops import unary_union
gdf_grouped.geometry = gdf_grouped.geometry.apply(unary_union)

#dissolve so we only keep the polygons that touch each other with the same cause and year 
gdf_exploded = gdf_grouped.explode()
print('After explode, shape : ', gdf_exploded.shape)
print(gdf_exploded.head())

In [15]:
gdf_exploded = gdf_exploded.reset_index()
gdf_exploded['year'] = gdf_exploded.year.apply(lambda x:str(int(x)))
gdf_exploded['cause'] = gdf_exploded.cause.astype(int)
gdf_exploded

,similar,level_1,index,year,index_right,cause,geometry
0,3-2018,0,1.913211e+06,2018,8.602171e+05,3,"POLYGON ((3327146.638 2782150.558, 3327146.638..."
1,3-2018,1,1.913211e+06,2018,8.602171e+05,3,"POLYGON ((3327116.638 2782210.558, 3327146.638..."
2,3-2018,2,1.913211e+06,2018,8.602171e+05,3,"POLYGON ((3326576.638 2782600.558, 3326666.638..."
3,3-2018,3,1.913211e+06,2018,8.602171e+05,3,"POLYGON ((3345626.638 2807350.558, 3345626.638..."
4,3-2018,4,1.913211e+06,2018,8.602171e+05,3,"POLYGON ((3345596.638 2807410.558, 3345626.638..."
...,...,...,...,...,...,...,...
5525510,2-2007,1682,4.154713e+06,2007,1.712784e+06,2,"POLYGON ((4270766.638 2201020.558, 4270736.638..."
5525511,2-2007,1683,4.154713e+06,2007,1.712784e+06,2,"POLYGON ((4270796.638 2201110.558, 4270856.638..."
5525512,2-2007,1684,4.154713e+06,2007,1.712784e+06,2,"POLYGON ((4269656.638 2201560.558, 4269686.638..."
5525513,2-2007,1685,4.154713e+06,2007,1.712784e+06,2,"POLYGON ((4269866.638 2201560.558, 4269896.638..."


In [22]:
import os

os.makedirs('../data/processed_datasets', exist_ok=True)
columns = ['year', 'cause', 'geometry']
gdf_exploded[columns].to_crs('epsg:4326').to_parquet(f'../data/processed_datasets/SenfSeidl_joined_EPSG4326.parquet')

## Checking map

In [1]:
import geopandas as gpd
processed = gpd.read_parquet('../data/processed_datasets/SenfSeidl_joined_EPSG4326.parquet')
raw = gpd.read_parquet('../data/SenfSeidl_maps/dgdf.parquet').to_crs('epsg:4326')

In [11]:
raw = gpd.read_parquet('../data/SenfSeidl_maps/dgdf.parquet').to_crs('epsg:4326')

There are less raws in the processed version. Good

In [12]:
#shape
print('raw shape : ', raw.shape)
print('processed shape : ', processed.shape)

raw shape :  (7028300, 6)
processed shape :  (5525515, 3)


## Adding tree species

In [2]:
import geopandas as gpd
processed = gpd.read_parquet('../data/processed_datasets/SenfSeidl_joined_EPSG4326_FR.parquet')
bdforet30 = gpd.read_parquet('../data/processed_datasets/BDFORET_EPSG2154_FR_simplified30.parquet').to_crs('epsg:4326')

In [7]:
bdforet30.sindex.query(processed.geometry[0], predicate='intersects')[0]

1677

In [17]:
def get_tree_species(row):
    index = bdforet30.sindex.query(row.geometry, predicate='intersects')
    if len(index) > 0:
        index = index[0]
        return bdforet30.iloc[index][['tree_type', 'essence']]
    else :
        return None, None

In [18]:
get_tree_species(processed.iloc[900])

tree_type    mixed
essence      mixed
Name: 1180032, dtype: object

In [26]:
#100000-> 1min 
# 5.5M -> 55min
processed.iloc[:100000].apply(get_tree_species, axis=1, result_type='expand')

,tree_type,essence
0,None,nc
1,None,nc
2,None,nc
3,mixed,mixed
4,mixed,mixed
...,...,...
99995,broadleaf,broadleaf
99996,broadleaf,broadleaf
99997,broadleaf,broadleaf
99998,broadleaf,broadleaf
